# TP2 Robotica Movel - Potential Fields

## Alunos
- Bernardo Nogueira Borges 2020006396
- Daniele Cassia Silva Diniz 2020076874


### Importando Libs

In [2]:
# from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import math
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import networkx as nx
import numpy as np

# client = RemoteAPIClient()
# sim = client.require('sim')
np.set_printoptions(precision=3,suppress=True)

### Definindo Parametros

### Obtendo Obstaculos da Imagem

### Criando Gradiente

### Movendo o Robo